In [72]:
import pdb
import pandas as pd
import numpy as np
import os
from scipy.stats import gaussian_kde
from datetime import datetime
from faker import Faker
import random
np.random.seed(3101)
random_state = np.random.RandomState(3101)
fake = Faker()

# Importing data:
bank1 = archive  
bank2 = bank+marketing

In [73]:
# print(os.getcwd())
bank_df_train = pd.read_csv('./data/main/train.csv')
bank_df_test = pd.read_csv('./data/main/test.csv')

In [74]:
print(bank_df_train.head())
print(len(bank_df_train))

# bank_df_train_clean = bank_df_train.drop_duplicates(subset="CustomerId", keep="first")
# print(len(bank_df_train_clean))

   id  CustomerId         Surname  CreditScore Geography Gender   Age  Tenure  \
0   0    15674932  Okwudilichukwu          668    France   Male  33.0       3   
1   1    15749177   Okwudiliolisa          627    France   Male  33.0       1   
2   2    15694510           Hsueh          678    France   Male  40.0      10   
3   3    15741417             Kao          581    France   Male  34.0       2   
4   4    15766172       Chiemenam          716     Spain   Male  33.0       5   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0       0.00              2        1.0             0.0        181449.97   
1       0.00              2        1.0             1.0         49503.50   
2       0.00              2        1.0             0.0        184866.69   
3  148882.54              1        1.0             1.0         84560.88   
4       0.00              2        1.0             1.0         15068.83   

   Exited  
0       0  
1       0  
2       0  
3       0  
4 

In [75]:
print(bank_df_test.head())
len(bank_df_test)
duplicates = bank_df_test.duplicated(subset=['CustomerId'], keep=False)
print(duplicates)
#no "exited" column

       id  CustomerId    Surname  CreditScore Geography  Gender   Age  Tenure  \
0  165034    15773898   Lucchese          586    France  Female  23.0       2   
1  165035    15782418       Nott          683    France  Female  46.0       2   
2  165036    15807120         K?          656    France  Female  34.0       7   
3  165037    15808905  O'Donnell          681    France    Male  36.0       8   
4  165038    15607314    Higgins          752   Germany    Male  38.0      10   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  
0       0.00              2        0.0             1.0        160976.75  
1       0.00              1        1.0             0.0         72549.27  
2       0.00              2        1.0             0.0        138882.09  
3       0.00              1        1.0             0.0        113931.57  
4  121263.62              1        1.0             0.0        139431.00  
0          True
1          True
2          True
3          True
4    

# Adding and populating features

### 13. Transaction History: Detailed transaction data offers insights into spending patterns and engagement. 
1 column for number of transactions in the past 3 months
1 column for amount transaction in the past 3 months

### 14. Customer Service Calls: Higher calls might indicate issues and dissatisfaction, affecting churn. 
Modeling with poisson distribution. Assuming that those who churned had more calls as they were more likely to had issues and thus needed calls. 
Poisson distribution deals with discrete events, which means it applies to situations where the number of occurrences (customer calls) can be whole numbers (0, 1, 2, 3, etc.). Customer calls happen independently throughout a specific time frame


In [76]:
from scipy.stats import poisson

avg_call_rate_per_year_unchurned = 12
avg_call_rate_per_year_churned = 24
# bank_df_train_clean['NumCalls'] = np.random.poisson(avg_call_rate_per_year, size = len(bank_df_train_clean))
bank_df_train['NumCalls'] = np.where(bank_df_train['Exited'] == 1, np.random.poisson(avg_call_rate_per_year_churned, size = len(bank_df_train)), np.random.poisson(avg_call_rate_per_year_unchurned, size = len(bank_df_train)))
print(bank_df_train.head())

(bank_df_train["NumCalls"]<0).any() #no negative calls


   id  CustomerId         Surname  CreditScore Geography Gender   Age  Tenure  \
0   0    15674932  Okwudilichukwu          668    France   Male  33.0       3   
1   1    15749177   Okwudiliolisa          627    France   Male  33.0       1   
2   2    15694510           Hsueh          678    France   Male  40.0      10   
3   3    15741417             Kao          581    France   Male  34.0       2   
4   4    15766172       Chiemenam          716     Spain   Male  33.0       5   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0       0.00              2        1.0             0.0        181449.97   
1       0.00              2        1.0             1.0         49503.50   
2       0.00              2        1.0             0.0        184866.69   
3  148882.54              1        1.0             1.0         84560.88   
4       0.00              2        1.0             1.0         15068.83   

   Exited  NumCalls  
0       0        15  
1       0        1

False

### 15. Total Relationship Count: Reflects the breadth of the customer's relationship with the bank. 
There are two main ways banks interpret total relationship count:

Number of Products and Services: This refers to the variety of products and services a customer uses from the bank. For example, if a customer has a checking account, savings account, and credit card with the same bank, their total relationship count based on products would be 3. A higher number indicates a broader and potentially more complex relationship. -> (sum of NumOfProducts + HasCrCard)

Total Assets Under Management: This focuses on the total value a customer entrusts to the bank. It considers the sum of their deposits (checking, savings), investments (stocks, bonds), and borrowings (loans, mortgages) managed through the bank. A higher total asset value suggests a stronger and more significant relationship for the bank. -> (we have deposit(balance). will need to create investments and borrowings). Currently GXS does not offer investment products so I will ignore that. 

In [77]:
mean_loan_amount = 5000  # Example mean loan amount
std_dev_loan_amount = 2000  # Example standard deviation

np.random.seed(3101)
bank_df_train['Loan_Amount'] = np.maximum(np.random.normal(mean_loan_amount, std_dev_loan_amount, len(bank_df_train)), 0)

print(bank_df_train.head())

   id  CustomerId         Surname  CreditScore Geography Gender   Age  Tenure  \
0   0    15674932  Okwudilichukwu          668    France   Male  33.0       3   
1   1    15749177   Okwudiliolisa          627    France   Male  33.0       1   
2   2    15694510           Hsueh          678    France   Male  40.0      10   
3   3    15741417             Kao          581    France   Male  34.0       2   
4   4    15766172       Chiemenam          716     Spain   Male  33.0       5   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0       0.00              2        1.0             0.0        181449.97   
1       0.00              2        1.0             1.0         49503.50   
2       0.00              2        1.0             0.0        184866.69   
3  148882.54              1        1.0             1.0         84560.88   
4       0.00              2        1.0             1.0         15068.83   

   Exited  NumCalls  Loan_Amount  
0       0        15  5993.2

### 24. Marketing Offers Accepted
Indicates responsiveness to incentives, affecting churn. Range 0-1, (percentage of marketing offers they accept, e.g.5 offer, 4 accepted, the value of the column 0.8)

In [78]:
marketing_df = pd.read_csv('./data/marketing offers/marketing_campaign.csv', sep=';')


marketing_df["MarketingOffersAcceptance"] = (marketing_df['AcceptedCmp1'] + marketing_df['AcceptedCmp2'] + marketing_df['AcceptedCmp3'] + marketing_df['AcceptedCmp4'] + marketing_df['AcceptedCmp5']) / 5
# print(marketing_df.head())

kde = gaussian_kde(marketing_df['MarketingOffersAcceptance'])

bank_df_train['MarketingOffersAcceptance'] = np.maximum(kde.resample(len(bank_df_train)).flatten(), 0)
bank_df_test['MarketingOffersAcceptance'] = np.maximum(kde.resample(len(bank_df_test)).flatten(), 0)

print(bank_df_train.head())

   id  CustomerId         Surname  CreditScore Geography Gender   Age  Tenure  \
0   0    15674932  Okwudilichukwu          668    France   Male  33.0       3   
1   1    15749177   Okwudiliolisa          627    France   Male  33.0       1   
2   2    15694510           Hsueh          678    France   Male  40.0      10   
3   3    15741417             Kao          581    France   Male  34.0       2   
4   4    15766172       Chiemenam          716     Spain   Male  33.0       5   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0       0.00              2        1.0             0.0        181449.97   
1       0.00              2        1.0             1.0         49503.50   
2       0.00              2        1.0             0.0        184866.69   
3  148882.54              1        1.0             1.0         84560.88   
4       0.00              2        1.0             1.0         15068.83   

   Exited  NumCalls  Loan_Amount  MarketingOffersAcceptance  


### 25. Channel Used for Transactions
Reflects preferred banking channels and engagement level. 
faker

In [79]:
transaction_channel_df = pd.read_csv('./data/main_payment_method/WA_Fn-UseC_-Telco-Customer-Churn.csv')

# print(transaction_channel_df.head())
methods = transaction_channel_df['PaymentMethod'].unique().tolist()
print(methods)
faked_data = [random.choice(methods) for _ in range(len(bank_df_train))]
faked_data_2 = [random.choice(methods) for _ in range(len(bank_df_test))]

bank_df_train['PaymentMethod'] = faked_data
bank_df_test['PaymentMethod'] = faked_data_2
print(bank_df_train.head())

['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)']
   id  CustomerId         Surname  CreditScore Geography Gender   Age  Tenure  \
0   0    15674932  Okwudilichukwu          668    France   Male  33.0       3   
1   1    15749177   Okwudiliolisa          627    France   Male  33.0       1   
2   2    15694510           Hsueh          678    France   Male  40.0      10   
3   3    15741417             Kao          581    France   Male  34.0       2   
4   4    15766172       Chiemenam          716     Spain   Male  33.0       5   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0       0.00              2        1.0             0.0        181449.97   
1       0.00              2        1.0             1.0         49503.50   
2       0.00              2        1.0             0.0        184866.69   
3  148882.54              1        1.0             1.0         84560.88   
4       0.00              2        1.0       

### 26. Customer Satisfaction Surveys
Provides direct feedback on satisfaction levels, predicting churn. 

In [80]:
cust_satisfaction_df = pd.read_csv('./data/satisfaction score/Customer-Churn-Records.csv')

kde = gaussian_kde(cust_satisfaction_df['Satisfaction Score'])

fake_data = np.maximum(kde.resample(len(bank_df_train)).flatten(), 0)
fake_data = np.minimum(fake_data.flatten(), 5)
fake_data = np.round(fake_data).astype(int)

fake_data_2 = np.maximum(kde.resample(len(bank_df_test)).flatten(), 0)
fake_data_2 = np.minimum(fake_data_2.flatten(), 5)
fake_data_2 = np.round(fake_data_2).astype(int)

bank_df_train['CustomerSatisfaction'] = fake_data
bank_df_test['CustomerSatisfaction'] = fake_data_2
# print(len(bank_df_test))
# print(len(fake_data_2))

print(bank_df_train['CustomerSatisfaction'].head(10))

0    5
1    3
2    5
3    5
4    5
5    5
6    5
7    3
8    4
9    3
Name: CustomerSatisfaction, dtype: int32


### 27. Income Source
Indicates financial stability and potential churn risk. 

In [81]:
income_source_df = pd.read_csv('./data/income source/adult.csv')

# print(transaction_channel_df.head())
income_sourcs = income_source_df['workclass'].unique().tolist()
income_sourcs.remove('?')
print(income_sourcs)
faked_data = [random.choice(income_sourcs) for _ in range(len(bank_df_train))]
faked_data_2 = [random.choice(income_sourcs) for _ in range(len(bank_df_test))]

bank_df_train['IncomeSource'] = faked_data
bank_df_test['IncomeSource'] = faked_data_2
print(bank_df_train.head())

['Private', 'Local-gov', 'Self-emp-not-inc', 'Federal-gov', 'State-gov', 'Self-emp-inc', 'Without-pay', 'Never-worked']
   id  CustomerId         Surname  CreditScore Geography Gender   Age  Tenure  \
0   0    15674932  Okwudilichukwu          668    France   Male  33.0       3   
1   1    15749177   Okwudiliolisa          627    France   Male  33.0       1   
2   2    15694510           Hsueh          678    France   Male  40.0      10   
3   3    15741417             Kao          581    France   Male  34.0       2   
4   4    15766172       Chiemenam          716     Spain   Male  33.0       5   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0       0.00              2        1.0             0.0        181449.97   
1       0.00              2        1.0             1.0         49503.50   
2       0.00              2        1.0             0.0        184866.69   
3  148882.54              1        1.0             1.0         84560.88   
4       0.00      

### 28. Credit Utilization
Reflects financial health and potential churn risk for credit customers.

In [82]:
credit_df = pd.read_csv('./data/credit utilization/BankChurners.csv')

kde = gaussian_kde(credit_df['Avg_Utilization_Ratio'])

fake_data = np.maximum(kde.resample(len(bank_df_train)).flatten(), 0)
fake_data = np.minimum(fake_data.flatten(), 1)
# fake_data = np.round(fake_data).astype(int)
fake_data_2 = np.maximum(kde.resample(len(bank_df_test)).flatten(), 0)
fake_data_2 = np.minimum(fake_data_2.flatten(), 1)

bank_df_train['CreditUtilization'] = fake_data
bank_df_test['CreditUtilization'] = fake_data_2

print(bank_df_train.head())

   id  CustomerId         Surname  CreditScore Geography Gender   Age  Tenure  \
0   0    15674932  Okwudilichukwu          668    France   Male  33.0       3   
1   1    15749177   Okwudiliolisa          627    France   Male  33.0       1   
2   2    15694510           Hsueh          678    France   Male  40.0      10   
3   3    15741417             Kao          581    France   Male  34.0       2   
4   4    15766172       Chiemenam          716     Spain   Male  33.0       5   

     Balance  NumOfProducts  ...  IsActiveMember  EstimatedSalary  Exited  \
0       0.00              2  ...             0.0        181449.97       0   
1       0.00              2  ...             1.0         49503.50       0   
2       0.00              2  ...             0.0        184866.69       0   
3  148882.54              1  ...             1.0         84560.88       0   
4       0.00              2  ...             1.0         15068.83       0   

   NumCalls  Loan_Amount  MarketingOffersAcceptanc

### 29. Response to Previous Retention Efforts
Records success or failure of previous retention efforts, guiding future strategies. 

In [83]:
retention_df = pd.read_csv('./data/retention/HR_comma_sep.csv')

kde = gaussian_kde(retention_df['promotion_last_5years'])

fake_data = np.maximum(kde.resample(len(bank_df_train)).flatten(), 0)
fake_data = np.minimum(fake_data.flatten(), 1)
fake_data = np.round(fake_data).astype(int)
fake_data_2 = np.maximum(kde.resample(len(bank_df_test)).flatten(), 0)
fake_data_2 = np.minimum(fake_data_2.flatten(), 1)
fake_data_2 = np.round(fake_data_2).astype(int)

bank_df_train['Retention'] = fake_data
bank_df_test['Retention'] = fake_data_2

print(bank_df_train.head())

   id  CustomerId         Surname  CreditScore Geography Gender   Age  Tenure  \
0   0    15674932  Okwudilichukwu          668    France   Male  33.0       3   
1   1    15749177   Okwudiliolisa          627    France   Male  33.0       1   
2   2    15694510           Hsueh          678    France   Male  40.0      10   
3   3    15741417             Kao          581    France   Male  34.0       2   
4   4    15766172       Chiemenam          716     Spain   Male  33.0       5   

     Balance  NumOfProducts  ...  EstimatedSalary  Exited  NumCalls  \
0       0.00              2  ...        181449.97       0        15   
1       0.00              2  ...         49503.50       0        14   
2       0.00              2  ...        184866.69       0        16   
3  148882.54              1  ...         84560.88       0        17   
4       0.00              2  ...         15068.83       0        10   

   Loan_Amount  MarketingOffersAcceptance              PaymentMethod  \
0  5993.267187

### 30. Change in behavior before n after
Average of percentage of increase/decrease (ranging from 0 - infinity, but most of the times it will be ard 0-2)

In [84]:
bank_df_train['ChanegInBehaviour'] = np.random.normal(2, 0.25, len(bank_df_train))
bank_df_test['ChanegInBehaviour']= np.random.normal(2, 0.25, len(bank_df_test))

### 31. Churn Date: The exact date the customer decided to exit.


In [85]:
start_date = "2023-01-01"
end_date = "2023-12-31"

random_dates = pd.date_range(start=start_date, end=end_date, freq='D')
random_dates_str = random_dates.strftime('%Y-%m-%d')
bank_df_train['ChurnDate'] = np.where(bank_df_train['Exited'] == 1, np.random.choice(random_dates_str) , np.nan) 
print(bank_df_train['ChurnDate'].head(10))

0           nan
1           nan
2           nan
3           nan
4           nan
5    2023-07-07
6           nan
7           nan
8           nan
9           nan
Name: ChurnDate, dtype: object


### 32. Sign Up Date: When did the customer create an account
has to be linked with churn date if any, and tenure. Tenure is usually rounded down, so 

In [86]:
def generate_signon_date(churn_date, tenure, exited):
    if exited == 0:
        churn_date = "2023-12-31"
    churn_date = pd.to_datetime(churn_date)
    max = churn_date - pd.DateOffset(years=tenure)
    min = churn_date - pd.DateOffset(years=(tenure + 1))
    random_dates = pd.date_range(min, max, freq='D').strftime('%Y-%m-%d')
    random_date = np.random.choice(random_dates)
    return random_date


bank_df_train['SignDate'] = bank_df_train.apply(lambda row: generate_signon_date(row['ChurnDate'], row['Tenure'], row['Exited']), axis = 1)
print(bank_df_train[['SignDate', 'ChurnDate', 'Tenure']].head(10))

bank_df_test['SignDate'] = bank_df_train.apply(lambda row: generate_signon_date("2023-12-31", row['Tenure'], 0), axis = 1)
# print(bank_df_train_clean.head(10))

     SignDate   ChurnDate  Tenure
0  2020-07-17         nan       3
1  2022-04-05         nan       1
2  2013-11-30         nan      10
3  2021-01-02         nan       2
4  2018-08-29         nan       5
5  2019-05-26  2023-07-07       4
6  2015-07-28         nan       8
7  2022-05-31         nan       1
8  2019-11-26         nan       4
9  2019-09-03         nan       4


# EDA
1. covarience matrix
2. LDA
3. apriori
4. domain knowledge

### 1. Covariance matrix

In [ ]:
#code for covariance matrix

### 2. LDA

In [ ]:
#code for LDA

### 3. Aprior

In [ ]:
#code for apriori

### 4. Domain knowledge

In [ ]:
#domain knowledge....

# Model training:
1. Random forest
2. XGBoost
3. Logistic regression